In [51]:
%reset -f
%config InteractiveShell.ast_node_interactivity = 'all'

# Fetching all Air Quality datasets into their dataframes
# Perform immediate concatenation per year
import os
import pandas as pd

# Initialize
dir = 'CAData/'
colidx = [0,1,2,4,5,6,11,15,17]     # column indexes to use
innerkeys = ['Date', 'Source', 'Site ID', 'STATE', 'COUNTY']
dataA2020 = pd.read_csv(dir + 'cf-2020-co.csv', parse_dates=True, usecols=colidx)
dataA2021 = pd.read_csv(dir + 'cf-2021-co.csv', parse_dates=True, usecols=colidx)
dataA2022 = pd.read_csv(dir + 'cf-2022-co.csv', parse_dates=True, usecols=colidx)

with os.scandir(dir) as datasets:
    for dataset in datasets:
        if dataset.is_file() and 'co' not in dataset.name:
            temp = pd.read_csv(dataset, parse_dates=True,
                               usecols=colidx)
            if '2020' in dataset.name:
                # cols_to_use = dataA2020.columns.difference(temp)
                pd.merge(dataA2020, temp, how='inner', on=innerkeys)
            elif '2021' in dataset.name:
                # cols_to_use = dataA2021.columns.difference(temp)
                pd.merge(dataA2021, temp, how='inner', on=innerkeys)
            elif '2022' in dataset.name:
                # cols_to_use = dataA2022.columns.difference(temp)
                pd.merge(dataA2022, temp, how='inner', on=innerkeys)

dataA2020
dataA2021
dataA2022

,Date,Source,Site ID,Daily Max 8-hour CO Concentration,UNITS_x,DAILY_AQI_VALUE_x,AQS_PARAMETER_DESC_x,STATE,COUNTY,Daily Max 1-hour NO2 Concentration,UNITS_y,DAILY_AQI_VALUE_y,AQS_PARAMETER_DESC_y
0,01/01/2020,AQS,60010009,0.7,ppm,8,Carbon monoxide,California,Alameda,21.3,ppb,20,Nitrogen dioxide (NO2)
1,01/02/2020,AQS,60010009,0.7,ppm,8,Carbon monoxide,California,Alameda,29.4,ppb,27,Nitrogen dioxide (NO2)
2,01/03/2020,AQS,60010009,0.9,ppm,10,Carbon monoxide,California,Alameda,31.9,ppb,29,Nitrogen dioxide (NO2)
3,01/04/2020,AQS,60010009,0.8,ppm,9,Carbon monoxide,California,Alameda,23.0,ppb,22,Nitrogen dioxide (NO2)
4,01/05/2020,AQS,60010009,0.7,ppm,8,Carbon monoxide,California,Alameda,22.9,ppb,21,Nitrogen dioxide (NO2)
...,...,...,...,...,...,...,...,...,...,...,...,...,...
22010,12/27/2020,AQS,60970004,0.5,ppm,6,Carbon monoxide,California,Sonoma,20.0,ppb,19,Nitrogen dioxide (NO2)
22011,12/28/2020,AQS,60970004,0.6,ppm,7,Carbon monoxide,California,Sonoma,15.7,ppb,14,Nitrogen dioxide (NO2)
22012,12/29/2020,AQS,60970004,0.5,ppm,6,Carbon monoxide,California,Sonoma,20.9,ppb,19,Nitrogen dioxide (NO2)
22013,12/30/2020,AQS,60970004,0.6,ppm,7,Carbon monoxide,California,Sonoma,10.3,ppb,9,Nitrogen dioxide (NO2)


,Date,Source,Site ID,Daily Max 8-hour CO Concentration,UNITS_x,DAILY_AQI_VALUE_x,AQS_PARAMETER_DESC_x,STATE,COUNTY,Daily Max 8-hour Ozone Concentration,UNITS_y,DAILY_AQI_VALUE_y,AQS_PARAMETER_DESC_y
0,01/01/2020,AQS,60010009,0.7,ppm,8,Carbon monoxide,California,Alameda,0.020,ppm,19,Ozone
1,01/02/2020,AQS,60010009,0.7,ppm,8,Carbon monoxide,California,Alameda,0.011,ppm,10,Ozone
2,01/03/2020,AQS,60010009,0.9,ppm,10,Carbon monoxide,California,Alameda,0.007,ppm,6,Ozone
3,01/04/2020,AQS,60010009,0.8,ppm,9,Carbon monoxide,California,Alameda,0.024,ppm,22,Ozone
4,01/05/2020,AQS,60010009,0.7,ppm,8,Carbon monoxide,California,Alameda,0.030,ppm,28,Ozone
...,...,...,...,...,...,...,...,...,...,...,...,...,...
18434,12/18/2020,AQS,60990005,0.8,ppm,9,Carbon monoxide,California,Stanislaus,0.020,ppm,19,Ozone
18435,12/19/2020,AQS,60990005,0.9,ppm,10,Carbon monoxide,California,Stanislaus,0.011,ppm,10,Ozone
18436,12/20/2020,AQS,60990005,0.5,ppm,6,Carbon monoxide,California,Stanislaus,0.014,ppm,13,Ozone
18437,12/21/2020,AQS,60990005,0.5,ppm,6,Carbon monoxide,California,Stanislaus,0.006,ppm,6,Ozone


,Date,Source,Site ID,Daily Max 8-hour CO Concentration,UNITS_x,DAILY_AQI_VALUE_x,AQS_PARAMETER_DESC_x,STATE,COUNTY,Daily Mean Pb Concentration,UNITS_y,DAILY_AQI_VALUE_y,AQS_PARAMETER_DESC_y
0,01/04/2020,AQS,60070008,0.5,ppm,6,Carbon monoxide,California,Butte,0.00135,ug/m3 SC,.,Lead (TSP) STP
1,01/16/2020,AQS,60070008,0.2,ppm,2,Carbon monoxide,California,Butte,0.00065,ug/m3 SC,.,Lead (TSP) STP
2,01/28/2020,AQS,60070008,0.3,ppm,3,Carbon monoxide,California,Butte,0.00065,ug/m3 SC,.,Lead (TSP) STP
3,02/09/2020,AQS,60070008,0.2,ppm,2,Carbon monoxide,California,Butte,0.00065,ug/m3 SC,.,Lead (TSP) STP
4,02/23/2020,AQS,60070008,0.3,ppm,3,Carbon monoxide,California,Butte,0.00174,ug/m3 SC,.,Lead (TSP) STP
...,...,...,...,...,...,...,...,...,...,...,...,...,...
543,02/09/2020,AQS,60850005,0.5,ppm,6,Carbon monoxide,California,Santa Clara,0.00376,ug/m3 SC,.,Lead (TSP) STP
544,02/21/2020,AQS,60850005,1.0,ppm,11,Carbon monoxide,California,Santa Clara,0.01040,ug/m3 SC,.,Lead (TSP) STP
545,03/04/2020,AQS,60850005,0.8,ppm,9,Carbon monoxide,California,Santa Clara,0.00596,ug/m3 SC,.,Lead (TSP) STP
546,03/16/2020,AQS,60850005,0.3,ppm,3,Carbon monoxide,California,Santa Clara,0.00065,ug/m3 SC,.,Lead (TSP) STP


,Date,Source,Site ID,Daily Max 8-hour CO Concentration,UNITS_x,DAILY_AQI_VALUE_x,AQS_PARAMETER_DESC_x,STATE,COUNTY,Daily Mean PM10 Concentration,UNITS_y,DAILY_AQI_VALUE_y,AQS_PARAMETER_DESC_y
0,01/01/2020,AQS,60070008,0.5,ppm,6,Carbon monoxide,California,Butte,27,ug/m3 SC,25,PM10 Total 0-10um STP
1,01/02/2020,AQS,60070008,0.6,ppm,7,Carbon monoxide,California,Butte,22,ug/m3 SC,20,PM10 Total 0-10um STP
2,01/03/2020,AQS,60070008,0.7,ppm,8,Carbon monoxide,California,Butte,30,ug/m3 SC,28,PM10 Total 0-10um STP
3,01/04/2020,AQS,60070008,0.5,ppm,6,Carbon monoxide,California,Butte,17,ug/m3 SC,16,PM10 Total 0-10um STP
4,01/05/2020,AQS,60070008,0.5,ppm,6,Carbon monoxide,California,Butte,18,ug/m3 SC,17,PM10 Total 0-10um STP
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8890,12/18/2020,AQS,60990005,0.8,ppm,9,Carbon monoxide,California,Stanislaus,25,ug/m3 SC,23,PM10 Total 0-10um STP
8891,12/19/2020,AQS,60990005,0.9,ppm,10,Carbon monoxide,California,Stanislaus,31,ug/m3 SC,29,PM10 Total 0-10um STP
8892,12/20/2020,AQS,60990005,0.5,ppm,6,Carbon monoxide,California,Stanislaus,29,ug/m3 SC,27,PM10 Total 0-10um STP
8893,12/21/2020,AQS,60990005,0.5,ppm,6,Carbon monoxide,California,Stanislaus,31,ug/m3 SC,29,PM10 Total 0-10um STP


,Date,Source,Site ID,Daily Max 8-hour CO Concentration,UNITS_x,DAILY_AQI_VALUE_x,AQS_PARAMETER_DESC_x,STATE,COUNTY,Daily Mean PM2.5 Concentration,UNITS_y,DAILY_AQI_VALUE_y,AQS_PARAMETER_DESC_y
0,01/01/2020,AQS,60010009,0.7,ppm,8,Carbon monoxide,California,Alameda,13.1,ug/m3 LC,53,PM2.5 - Local Conditions
1,01/02/2020,AQS,60010009,0.7,ppm,8,Carbon monoxide,California,Alameda,7.6,ug/m3 LC,32,PM2.5 - Local Conditions
2,01/03/2020,AQS,60010009,0.9,ppm,10,Carbon monoxide,California,Alameda,15.0,ug/m3 LC,57,PM2.5 - Local Conditions
3,01/04/2020,AQS,60010009,0.8,ppm,9,Carbon monoxide,California,Alameda,9.5,ug/m3 LC,40,PM2.5 - Local Conditions
4,01/05/2020,AQS,60010009,0.7,ppm,8,Carbon monoxide,California,Alameda,6.1,ug/m3 LC,25,PM2.5 - Local Conditions
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21931,12/18/2020,AQS,60990005,0.8,ppm,9,Carbon monoxide,California,Stanislaus,15.2,ug/m3 LC,58,PM2.5 - Local Conditions
21932,12/19/2020,AQS,60990005,0.9,ppm,10,Carbon monoxide,California,Stanislaus,26.9,ug/m3 LC,82,PM2.5 - Local Conditions
21933,12/20/2020,AQS,60990005,0.5,ppm,6,Carbon monoxide,California,Stanislaus,25.5,ug/m3 LC,79,PM2.5 - Local Conditions
21934,12/21/2020,AQS,60990005,0.5,ppm,6,Carbon monoxide,California,Stanislaus,27.0,ug/m3 LC,82,PM2.5 - Local Conditions


...

,Date,Source,Site ID,Daily Max 8-hour CO Concentration,UNITS_x,DAILY_AQI_VALUE_x,AQS_PARAMETER_DESC_x,STATE,COUNTY,Daily Mean PM2.5 Concentration,UNITS_y,DAILY_AQI_VALUE_y,AQS_PARAMETER_DESC_y
0,01/01/2022,AQS,60010009,0.7,ppm,8,Carbon monoxide,California,Alameda,13.2,ug/m3 LC,53,PM2.5 - Local Conditions
1,01/02/2022,AQS,60010009,0.6,ppm,7,Carbon monoxide,California,Alameda,8.2,ug/m3 LC,34,PM2.5 - Local Conditions
2,01/03/2022,AQS,60010009,0.6,ppm,7,Carbon monoxide,California,Alameda,3.8,ug/m3 LC,16,PM2.5 - Local Conditions
3,01/04/2022,AQS,60010009,0.4,ppm,5,Carbon monoxide,California,Alameda,3.1,ug/m3 LC,13,PM2.5 - Local Conditions
4,01/05/2022,AQS,60010009,0.4,ppm,5,Carbon monoxide,California,Alameda,4.7,ug/m3 LC,20,PM2.5 - Local Conditions
...,...,...,...,...,...,...,...,...,...,...,...,...,...
18173,12/18/2022,AQS,60990005,0.9,ppm,10,Carbon monoxide,California,Stanislaus,27.4,ug/m3 LC,83,PM2.5 - Local Conditions
18174,12/19/2022,AQS,60990005,0.2,ppm,2,Carbon monoxide,California,Stanislaus,27.2,ug/m3 LC,83,PM2.5 - Local Conditions
18175,12/19/2022,AQS,60990005,0.2,ppm,2,Carbon monoxide,California,Stanislaus,23.5,ug/m3 LC,75,Acceptable PM2.5 AQI & Speciation Mass
18176,12/20/2022,AQS,60990005,0.4,ppm,5,Carbon monoxide,California,Stanislaus,32.7,ug/m3 LC,94,PM2.5 - Local Conditions


,Date,Source,Site ID,Daily Max 8-hour CO Concentration,UNITS_x,DAILY_AQI_VALUE_x,AQS_PARAMETER_DESC_x,STATE,COUNTY,Daily Max 1-hour SO2 Concentration,UNITS_y,DAILY_AQI_VALUE_y,AQS_PARAMETER_DESC_y
0,01/01/2022,AQS,60010011,0.9,ppm,10,Carbon monoxide,California,Alameda,0.1,ppb,0,Sulfur dioxide
1,01/02/2022,AQS,60010011,1.2,ppm,14,Carbon monoxide,California,Alameda,0.5,ppb,0,Sulfur dioxide
2,01/03/2022,AQS,60010011,1.1,ppm,13,Carbon monoxide,California,Alameda,0.0,ppb,0,Sulfur dioxide
3,01/04/2022,AQS,60010011,0.5,ppm,6,Carbon monoxide,California,Alameda,-0.3,ppb,0,Sulfur dioxide
4,01/05/2022,AQS,60010011,0.5,ppm,6,Carbon monoxide,California,Alameda,-0.4,ppb,0,Sulfur dioxide
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6059,12/27/2022,AQS,60950004,0.5,ppm,6,Carbon monoxide,California,Solano,0.5,ppb,0,Sulfur dioxide
6060,12/28/2022,AQS,60950004,0.9,ppm,10,Carbon monoxide,California,Solano,0.9,ppb,0,Sulfur dioxide
6061,12/29/2022,AQS,60950004,0.6,ppm,7,Carbon monoxide,California,Solano,0.4,ppb,0,Sulfur dioxide
6062,12/30/2022,AQS,60950004,0.4,ppm,5,Carbon monoxide,California,Solano,2.1,ppb,3,Sulfur dioxide


,Date,Source,Site ID,Daily Max 8-hour CO Concentration,UNITS,DAILY_AQI_VALUE,AQS_PARAMETER_DESC,STATE,COUNTY
0,01/01/2020,AQS,60010009,0.7,ppm,8,Carbon monoxide,California,Alameda
1,01/02/2020,AQS,60010009,0.7,ppm,8,Carbon monoxide,California,Alameda
2,01/03/2020,AQS,60010009,0.9,ppm,10,Carbon monoxide,California,Alameda
3,01/04/2020,AQS,60010009,0.8,ppm,9,Carbon monoxide,California,Alameda
4,01/05/2020,AQS,60010009,0.7,ppm,8,Carbon monoxide,California,Alameda
...,...,...,...,...,...,...,...,...,...
22121,12/18/2020,AQS,60990005,0.8,ppm,9,Carbon monoxide,California,Stanislaus
22122,12/19/2020,AQS,60990005,0.9,ppm,10,Carbon monoxide,California,Stanislaus
22123,12/20/2020,AQS,60990005,0.5,ppm,6,Carbon monoxide,California,Stanislaus
22124,12/21/2020,AQS,60990005,0.5,ppm,6,Carbon monoxide,California,Stanislaus


,Date,Source,Site ID,Daily Max 8-hour CO Concentration,UNITS,DAILY_AQI_VALUE,AQS_PARAMETER_DESC,STATE,COUNTY
0,01/01/2021,AQS,60010009,0.7,ppm,8,Carbon monoxide,California,Alameda
1,01/02/2021,AQS,60010009,0.7,ppm,8,Carbon monoxide,California,Alameda
2,01/03/2021,AQS,60010009,0.7,ppm,8,Carbon monoxide,California,Alameda
3,01/04/2021,AQS,60010009,0.7,ppm,8,Carbon monoxide,California,Alameda
4,01/05/2021,AQS,60010009,0.6,ppm,7,Carbon monoxide,California,Alameda
...,...,...,...,...,...,...,...,...,...
20753,12/27/2021,AQS,60990005,0.3,ppm,3,Carbon monoxide,California,Stanislaus
20754,12/28/2021,AQS,60990005,0.4,ppm,5,Carbon monoxide,California,Stanislaus
20755,12/29/2021,AQS,60990005,0.3,ppm,3,Carbon monoxide,California,Stanislaus
20756,12/30/2021,AQS,60990005,0.4,ppm,5,Carbon monoxide,California,Stanislaus


,Date,Source,Site ID,Daily Max 8-hour CO Concentration,UNITS,DAILY_AQI_VALUE,AQS_PARAMETER_DESC,STATE,COUNTY
0,01/01/2022,AQS,60010009,0.7,ppm,8,Carbon monoxide,California,Alameda
1,01/02/2022,AQS,60010009,0.6,ppm,7,Carbon monoxide,California,Alameda
2,01/03/2022,AQS,60010009,0.6,ppm,7,Carbon monoxide,California,Alameda
3,01/04/2022,AQS,60010009,0.4,ppm,5,Carbon monoxide,California,Alameda
4,01/05/2022,AQS,60010009,0.4,ppm,5,Carbon monoxide,California,Alameda
...,...,...,...,...,...,...,...,...,...
18529,12/27/2022,AQS,60990005,0.3,ppm,3,Carbon monoxide,California,Stanislaus
18530,12/28/2022,AQS,60990005,0.6,ppm,7,Carbon monoxide,California,Stanislaus
18531,12/29/2022,AQS,60990005,0.4,ppm,5,Carbon monoxide,California,Stanislaus
18532,12/30/2022,AQS,60990005,0.2,ppm,2,Carbon monoxide,California,Stanislaus
